<a href="https://colab.research.google.com/github/Tanish-Sarkar/Elite-Transformers/blob/main/Module5%20-%20Applications%20%26%20Variants/decoder_only_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Import the modules**

In [1]:
# try:
#     import pytorch_lightning as L
# except ImportError:
#     !pip install pytorch-lightning
#     import pytorch_lightning as L

In [2]:
!pip install lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.0/846.0 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 30.1 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
import lightning as L

## **Create the input and output and data**

In [4]:
token_to_id = {'what' : 0,
               'is' : 1,
               'statquest' : 2,
               'awesome': 3,
               '<EOS>' : 4, ## <EOS> = end of sequence
              }

id_to_token = dict(map(reversed, token_to_id.items()))

In [5]:
inputs = torch.tensor([[token_to_id["what"], ## input #1: what is statquest <EOS> awesome
                        token_to_id["is"],
                        token_to_id["statquest"],
                        token_to_id["<EOS>"],
                        token_to_id["awesome"]],

                       [token_to_id["statquest"], # input #2: statquest is what <EOS> awesome
                        token_to_id["is"],
                        token_to_id["what"],
                        token_to_id["<EOS>"],
                        token_to_id["awesome"]]])

In [6]:
labels = torch.tensor([[token_to_id["is"],
                        token_to_id["statquest"],
                        token_to_id["<EOS>"],
                        token_to_id["awesome"],
                        token_to_id["<EOS>"]],

                       [token_to_id["is"],
                        token_to_id["what"],
                        token_to_id["<EOS>"],
                        token_to_id["awesome"],
                        token_to_id["<EOS>"]]])

### Packaging everything up into a DataLoader


In [7]:
dataset = TensorDataset(inputs, labels)
dataloader = DataLoader(dataset)

## **Position Encoding**

In [8]:
class PositionEncoding(nn.Module):
  def __init__(self, d_model=2, max_len=6):
    super().__init__()
    pe = torch.zeros(max_len, d_model)
    position = torch.arange(0, max_len, 1).float().unsqueeze(1)
    embedding_index = torch.arange(start=0, end=d_model, step=2).float()
    div_term = 1/torch.tensor(10000.0)**(embedding_index / d_model)
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    self.register_buffer('pe', pe)

  def forward(self, word_embedding):
    return word_embedding + self.pe[:word_embedding.size(0), :]

## **Attention**

In [9]:
class Attention(nn.Module):
  def __init__(self, d_model=2):
    super().__init__()
    self.d_model = d_model

    self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
    self.W_k = nn.Linear(in_features=d_model, out_features=d_model, bias=False)
    self.W_v = nn.Linear(in_features=d_model, out_features=d_model, bias=False)

    self.row_dim = 0
    self.col_dim = 1

  def forward(self, encodings_for_q, encodings_for_k, encodings_for_v, mask=None):
    q = self.W_q(encodings_for_q)
    k = self.W_k(encodings_for_k)
    v = self.W_v(encodings_for_v)

    sims = torch.matmul(q, k.transpose(dim0=self.row_dim, dim1=self.col_dim))
    scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)

    if mask is not None:
      scaled_sims = scaled_sims.masked_fill(mask, -1e9)

    attention_precents = F.softmax(scaled_sims, dim=self.col_dim)
    attention_scores = torch.matmul(attention_precents, v)
    return attention_scores

# **The Decoder-Only Transformer**

In [10]:
class DecoderOnlyTransformer(L.LightningModule):

    def __init__(self, num_tokens=4, d_model=2, max_len=6):
      super().__init__()
      self.we = nn.Embedding(num_embeddings=num_tokens, embedding_dim=d_model)
      self.pe = PositionEncoding(d_model=d_model, max_len=max_len)
      self.self_attention = Attention(d_model=d_model)
      self.fc_layer = nn.Linear(in_features=d_model, out_features=num_tokens)
      self.loss = nn.CrossEntropyLoss()

    def forward(self, token_ids):
      word_embeddings = self.we(token_ids)
      position_encoded = self.pe(word_embeddings)
      mask = torch.tril(torch.ones(token_ids.size(dim=0), token_ids.size(dim=0), device=self.device))

      mask = mask == 0

      self_attention_values = self.self_attention(position_encoded, position_encoded, position_encoded, mask=mask)
      residual_connection_values = position_encoded + self_attention_values
      fc_layer_output = self.fc_layer(residual_connection_values)

      return fc_layer_output


    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.1)


    def training_step(self, batch, batch_idx):
        input_tokens, labels = batch # collect input
        output = self.forward(input_tokens[0])
        loss = self.loss(output, labels[0])

        return loss

In [11]:
model = DecoderOnlyTransformer(num_tokens=len(token_to_id), d_model=2, max_len=6)
model_input = torch.tensor([token_to_id["what"],
                            token_to_id["is"],
                            token_to_id["statquest"],
                            token_to_id["<EOS>"]])
input_length = model_input.size(dim=0)

predictions = model(model_input)
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
predicted_ids = predicted_id # Corrected: Changed 'prediction_ids' to 'predicted_ids'

max_length= 6
for i in range(input_length, max_length):
    if (predicted_id == token_to_id["<EOS>"]): # if the prediction is <EOS>, then we are done
        break

    model_input = torch.cat((model_input, predicted_id))

    predictions = model(model_input)
    predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
    predicted_ids = torch.cat((predicted_ids, predicted_id))

## Now printout the predicted output phrase.
print("Predicted Tokens:\n")
for id in predicted_ids:
    print("\t", id_to_token[id.item()])

Predicted Tokens:

	 is
	 is
	 is


## **Train the Decoder-Only Transformer**

In [13]:
trainer = L.Trainer(max_epochs=30)
trainer.fit(model, train_dataloaders=dataloader)

INFO: 💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name           ┃ Type             ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ we             │ Embedding        │     10 │ train │     0 │
│ 1 │ pe             │ PositionEncoding │      0 │ train │     0 │
│ 2 │ self_attention │ Attention        │     12 │ train │     0 │
│ 3 │ fc_layer       │ Linear           │     15 │ train │     0 │
│ 4 │ loss           │ CrossEntropyLoss │      0 │ train │     0 │
└───┴────────────────┴──────────────────┴────────┴───────┴───────┘

Trainable params: 37                                                                                               
Non-trainable params: 0                                                                                            
Total params: 37                                                                                                   
Total estimated model params size (MB): 0                                                                          
Modules in train mode: 8                                                                                           
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

/usr/local/lib/python3.12/dist-packages/lightning/pytorch/loops/fit_loop.py:317: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
INFO: `Trainer.fit` stopped: `max_epochs=30` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


## **Use the Trained Transformer**

In [14]:

model_input = torch.tensor([token_to_id["what"],
                            token_to_id["is"],
                            token_to_id["statquest"],
                            token_to_id["<EOS>"]])
input_length = model_input.size(dim=0)

predictions = model(model_input)
predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
predicted_ids = predicted_id

for i in range(input_length, max_length):
  if (predicted_id == token_to_id["<EOS>"]): # if the prediction is <EOS>, then we will stop
    break

  model_input = torch.cat((model_input, predicted_id))

  predictions = model(model_input)
  predicted_id = torch.tensor([torch.argmax(predictions[-1,:])])
  predicted_ids = torch.cat((predicted_ids, predicted_id))

print("Predicted Tokens:\n")
for id in predicted_ids:
  print("\t", id_to_token[id.item()])

Predicted Tokens:

	 awesome
	 <EOS>
